<a href="https://colab.research.google.com/github/fabrytp8/AG1/blob/main/ej/testing_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import pickle
from pprint import pprint
from skimage.feature import hog

from sklearn import preprocessing
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier

from tabulate import tabulate
from sklearn import metrics

In [ ]:
# Cargar todos los objetos obtenidos en el entrenamiento
# estandarizador
with open('objetos/estandarizador.pickle', 'rb') as fr:
    std = pickle.load(fr)
    
# selector
with open('objetos/selector.pickle', 'rb') as fr:
    sel = pickle.load(fr)

# model
with open('objetos/clasificador.pickle', 'rb') as fr:
    model = pickle.load(fr)


In [ ]:
# Craga de datos de test
images, target = [],[]
folder = os.listdir('test/')
# print(folder)
for i in np.arange(len(folder)):
    name = folder[i] #name image
    img = cv2.imread('test/' + name, 0)
    img[img<255]=0
    images.append(img)
    
    idx = name.find('_')
    tgt = int(name[idx+1])
    target.append(tgt)

images = np.array(images)
target = np.array(target)
print(np.shape(images))
print(np.shape(target))

(320, 16, 16)
(320,)


In [ ]:
# 5) Extracciónd e características de test
X_test = []
y_test = target
for i in np.arange(len(folder)):
    features = hog(images[i],orientations=9, pixels_per_cell=(4,4), cells_per_block=(2,2),
                  transform_sqrt = True, block_norm='L1')
    X_test.append(features)
X_test = np.array(X_test)
print(np.shape(X_test))
print(np.shape(y_test))

(320, 324)
(320,)


In [ ]:
# 6) Estandarización y caracterizaciónd e test
X_test = std.transform(X_test)
print(np.shape(X_test))

(320, 324)


In [ ]:
# 7) Selección de características 
X_test = sel.transform(X_test)
print(np.shape(X_test))

(320, 144)


In [ ]:
# 8) Extraer predicciones
y_pred = model.predict(X_test)
print(np.shape(y_pred))

(320,)


In [ ]:
# 9)Evaluar resultados
from tabulate import tabulate
from sklearn import metrics

headers = ['','CLF']
P,S,FS,ACC = [['Precision'],['Sensibilidad'],['F1-score'],['Accuracy']]

P.append(np.round(metrics.precision_score(y_test, y_pred, average = 'macro'),4))
S.append(np.round(metrics.recall_score(y_test, y_pred, average = 'macro'),4))
FS.append(np.round(metrics.f1_score(y_test, y_pred, average = 'macro'),4))
ACC.append(np.round(metrics.accuracy_score(y_test, y_pred),4))
my_data = [tuple(P), tuple(S), tuple(FS), tuple(ACC)]
print(tabulate(my_data, headers=headers))

# Confusion matrix
print('\nCONFUSION MATRIX')
print(metrics.confusion_matrix(y_test, y_pred, normalize=None, ))

                 CLF
------------  ------
Precision     0.861
Sensibilidad  0.8438
F1-score      0.8414
Accuracy      0.8438

CONFUSION MATRIX
[[31  1  0  0  0  0  0  0  0  0]
 [ 0 31  1  0  0  0  0  0  0  0]
 [ 2  2 26  0  0  0  0  2  0  0]
 [ 1  5  0 20  0  3  0  0  3  0]
 [ 0  0  2  0 30  0  0  0  0  0]
 [ 0  3  0  0  0 29  0  0  0  0]
 [ 3  0  0  0  0  0 29  0  0  0]
 [ 0  1  0  0  0  0  0 31  0  0]
 [ 2  1  0  0  0  0  7  0 21  1]
 [ 2  1  0  1  0  3  1  0  2 22]]
